# Network Anomaly Detection

This notebook runs through an example project for creating an anomaly detection system for network data.

In [1]:
import time
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import ruptures as rpt
import seaborn as sns
from adtk.data import validate_series
from adtk.detector import LevelShiftAD, PersistAD, SeasonalAD
from adtk.transformer import ClassicSeasonalDecomposition
from adtk.visualization import plot
from plotly.subplots import make_subplots
from utils import combine_anomalies, cusum_ad, pelt_ad, plot_anomalies, plot_network_metrics

warnings.filterwarnings("ignore")


## Data Generation

The network timeseries data is assumed to be telemetry data simplified as aggregated metrics over time: 

`timestamp | bytes_in | bytes_out | error_rate`

The data is aggregated to 5 minute intervals.


In [2]:
def generate_data(
        start='2026-02-01', 
        end='2026-02-02', 
        base_bytes=1000,
        peak_bytes=500,
        peak_hour=21
        ):
    
    # Get timeseries of 5 min timestamps
    ts = pd.date_range(start, end, freq='5min')
    
    # Convert to decimal values within the day
    ts_decimal = ts.hour + (ts.minute / 60) + (ts.second / 3600)

    # Create a cosine cycle to simulate patterns over the day, peaking at the specified hour
    cycle = np.cos(2 * np.pi * (ts_decimal - peak_hour) / 24)

    # Calculate bytes_in and bytes_out with some randomness
    bytes_in = np.maximum(base_bytes + (peak_bytes * cycle) + np.random.normal(0, 100, len(ts)), 0)
    bytes_out = np.maximum((bytes_in / 3) - np.random.normal(0, 50, len(ts)), 0)

    # Add random error rates with a baseline level (and a slight increase during peak hours)
    error_rate = np.minimum(np.maximum(0.05 + (0.01 * cycle) + np.random.normal(0, 0.02, len(ts)), 0), 1)

    # Create a dataframe
    df = pd.DataFrame(index = ts, data={
        'bytes_in': bytes_in,
        'bytes_out': bytes_out,
        'error_rate': error_rate
    })

    return df



In [3]:
df = generate_data('2026-02-02', '2026-02-08')
plot_network_metrics(df, title='Clean network traffic')

### Injecting anomalies

There are some different types of anomalies we may want to detect:
- Point anomalies - suddens spikes or drops
- Level shifts - protracted increases or decreases in the level of the data
- Trend changes - for example from flat to some linear increase

In [4]:
def generate_point_anomalies(df, num_anomalies=10):
    """
    Randomly add spikes or drops in the metrics
    """
    df_anomalous = df.copy()
    anomaly_indices = np.random.choice(df.index, size=num_anomalies, replace=False)

    anomaly_flags = pd.DataFrame(index=df.index, columns=['bytes_in', 'bytes_out', 'error_rate'])
    
    for idx in anomaly_indices:
        # Randomly decide the type of anomaly
        anomaly_type = np.random.choice(['spike', 'drop', 'error_spike'])
        
        if anomaly_type == 'spike':
            df_anomalous.at[idx, 'bytes_in'] *= np.random.uniform(2, 3)  # Spike in bytes_in
            df_anomalous.at[idx, 'bytes_out'] *= np.random.uniform(2, 3)  # Spike in bytes_out
            anomaly_flags.at[idx, 'bytes_in'] = True
            anomaly_flags.at[idx, 'bytes_out'] = True
            
        elif anomaly_type == 'drop':
            df_anomalous.at[idx, 'bytes_in'] *= np.random.uniform(0.1, 0.2)  # Drop in bytes_in
            df_anomalous.at[idx, 'bytes_out'] *= np.random.uniform(0.1, 0.2)  # Drop in bytes_out
            anomaly_flags.at[idx, 'bytes_in'] = True
            anomaly_flags.at[idx, 'bytes_out'] = True
            
        # Note that in reality we might expect spikes and drops in traffic to correlate with spikes in error rates
        elif anomaly_type == 'error_spike':
            df_anomalous.at[idx, 'error_rate'] = min(df_anomalous.at[idx, 'error_rate'] + np.random.uniform(0.1, 0.5), 1)  # Spike in error rate
            anomaly_flags.at[idx, 'error_rate'] = True
    
    return df_anomalous, anomaly_flags.fillna(False)

In [5]:
df_point_anomalies, point_anomaly_indices = generate_point_anomalies(df)
plot_network_metrics(df_point_anomalies, title='Network traffic with point anomalies')

In [293]:
def generate_level_anomalies(df, num_anomalies=1):
    """
    Randomly add sustained periods of increased traffic and error rates.
    The function ensures that level change periods do not overlap.
    Up to `num_anomalies` periods will be added, but this may be reduced if periods overlap and are stripped out.
    """
    df_anomalous = df.copy()
    
    anomaly_flags = pd.DataFrame(index=df.index, columns=['bytes_in', 'bytes_out', 'error_rate'])
    
    # Track which positions are used (including padding)
    used_positions = set()
    padding = 6
    
    for i in range(num_anomalies):    
        # Get available positions throughout the first third of the timeseries (not in used set)
        available_positions = [p for p in range(int(len(df)//3)) if p not in used_positions]
        
        # Stop if not enough positions available
        if len(available_positions) < 6:
            break
        
        # Pick random start from available positions
        start_pos = np.random.choice(available_positions)
        length = np.random.randint(48, 288*2)  # Anomalies lasting between 4 hours and 2 days
        
        # Calculate the actual range that will be marked as used
        end_pos = min(start_pos + length, len(df))
        start_remove = max(0, start_pos - padding)
        end_remove = min(end_pos + padding, len(df))
        
        # Check if this range overlaps with already used positions
        range_to_use = set(range(start_remove, end_remove))
        if range_to_use & used_positions:  # If there's any intersection, skip this anomaly
            continue
        
        level = np.random.uniform(1.2, 1.5)  # Anomaly level (e.g., 1.2x to 1.5x increase)
        
        # Get positional indices for the anomaly period
        anomaly_positions = np.arange(start_pos, end_pos)
        
        # Convert positions to actual index labels for DataFrame operations
        anomaly_indices = df.index[anomaly_positions]
        
        df_anomalous.loc[anomaly_indices, 'bytes_in'] *= level
        df_anomalous.loc[anomaly_indices, 'bytes_out'] *= level
        df_anomalous.loc[anomaly_indices, 'error_rate'] = np.minimum(df_anomalous.loc[anomaly_indices, 'error_rate'] + (level * 0.05), 1)
        
        # Mark the padded range as used
        used_positions.update(range_to_use)
        
        # Mark the start and end of the level change
        anomaly_flags.loc[df.index[start_pos], ['bytes_in', 'bytes_out', 'error_rate']] = True
        anomaly_flags.loc[df.index[np.minimum(end_pos, len(df)-1)], ['bytes_in', 'bytes_out', 'error_rate']] = True
    
    return df_anomalous, anomaly_flags.fillna(False)

In [307]:
df_level_anomalies, level_anomaly_indices = generate_level_anomalies(df)
plot_network_metrics(df_level_anomalies, title='Network traffic with level anomalies')

In [308]:
def generate_trend_anomalies(df):
    """
    Create a change in the underlying trend of the data.

    """

    df_anomalous = df.copy()

    anomaly_flags = pd.DataFrame(index=df.index, columns=['bytes_in', 'bytes_out', 'error_rate'])

    # Allow trend change starts in the second half of the series
    start = np.random.choice(range(len(df)//2, int(len(df)//1.1)))
    length = len(df) - start

    anomaly_indices = df.index[start:start+length]

    m = np.random.uniform(0.01, 0.02) # Random gradient increase of 1 to 2 percentage points per time step

    for i, pos in enumerate(anomaly_indices):
        # df_anomalous.at[pos, 'bytes_in'] *= (1 + (m * i))
        # df_anomalous.at[pos, 'bytes_out'] *= (1 + (m * i))
        df_anomalous.at[pos, 'error_rate'] = np.minimum(df_anomalous.at[pos, 'error_rate'] + ((m * i) * 0.03), 1) # Scaling factor for error_rate increase due to lower base values
        if i == 0:
            anomaly_flags.loc[pos, 'error_rate'] = True

    return df_anomalous, anomaly_flags.fillna(False)



In [317]:
df_trend_anomalies, trend_anomaly_indices = generate_trend_anomalies(df)
plot_network_metrics(df_trend_anomalies, title='Network traffic with trend anomalies')

In [327]:
# Combine effects of all anomalies
df_all_anomalies = df.copy()
df_all_anomalies, point_anomaly_indices = generate_point_anomalies(df_all_anomalies)
df_all_anomalies, level_anomaly_indices = generate_level_anomalies(df_all_anomalies)
df_all_anomalies, trend_anomaly_indices = generate_trend_anomalies(df_all_anomalies)

plot_network_metrics(df_all_anomalies, title='Network traffic with all anomalies')


## Anomaly Detection

Using a suite of approaches to detect each of the anomaly types: 

- Point anomalies - Seasonal Anomaly Detector
- Level anomalies - Level Shift Detector
- Trend anomalies - CUSUM and Pelt (from ruptures)


In [329]:
# Check for consistency and absence on duplicates in the timeseries
dfts = validate_series(df_all_anomalies)

In [330]:
persist_ad = PersistAD(window=5, c=5)
persist_ad_anomalies = persist_ad.fit_detect(dfts).fillna(False)
plot_anomalies(dfts, persist_ad_anomalies, title='Persist AD Test Detector')

In [332]:
seasonal_ad = SeasonalAD(c=3, freq=288)  # Assuming daily seasonality with 5-minute intervals (288 intervals per day)
seasonal_anomalies = seasonal_ad.fit_detect(dfts).fillna(False)
plot_anomalies(dfts, seasonal_anomalies, title='Seasonal Anomaly Detector')

In [ ]:
levelshift_ad = LevelShiftAD(window=10, c=4)
levelshift_anomalies = levelshift_ad.fit_detect(dfts).fillna(False)
plot_anomalies(dfts, levelshift_anomalies, title='Level Shift Detector')

In [334]:
pelt_anomalies = pelt_ad(dfts, penalty=20, seasonal_decomposition=True)
plot_anomalies(dfts, pelt_anomalies, title='PELT Trend Change Detection')

In [337]:
pred_combined_anomalies = combine_anomalies([persist_ad_anomalies, seasonal_anomalies, levelshift_anomalies, pelt_anomalies])
plot_anomalies(dfts, pred_combined_anomalies, title='Combined Anomaly Detectors')

## Evaluation of Detection Performance

In [ ]:
combine_anomalies()

point_anomaly_indices
level_anomaly_indices
trend_anomaly_indices

,bytes_in,bytes_out,error_rate
2026-02-02 00:00:00,False,False,False
2026-02-02 00:05:00,False,False,False
2026-02-02 00:10:00,False,False,False
2026-02-02 00:15:00,False,False,False
2026-02-02 00:20:00,False,False,False
...,...,...,...
2026-02-07 23:40:00,False,False,False
2026-02-07 23:45:00,False,False,False
2026-02-07 23:50:00,False,False,False
2026-02-07 23:55:00,False,False,False


array([  89, 1334,  780,  757,  182, 1429,    7,  158, 1075,  376])